In [2]:
# Enhanced Imports
from keras import models, layers, regularizers, callbacks
from keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt
import os, warnings
from PIL import Image
import pandas as pd
import keras

# Suppress Warnings for Cleaner Output
warnings.simplefilter('ignore')

# Data Acquisition Function
def load_data(file_paths):
    return [np.load(path) for path in file_paths]

# Dataset File Paths
data_paths = [
    'C:/Datasets/eda-and-data-preprocessing/train_images.npy',
    'C:/Datasets/eda-and-data-preprocessing/train_labels.npy',
    'C:/Datasets/eda-and-data-preprocessing/train_images_third.npy',
    'C:/Datasets/eda-and-data-preprocessing/train_labels_third.npy',
    'C:/Datasets/eda-and-data-preprocessing/val_images.npy',
    'C:/Datasets/eda-and-data-preprocessing/val_labels.npy',
    'C:/Datasets/eda-and-data-preprocessing/test_images.npy',
    'C:/Datasets/eda-and-data-preprocessing/test_labels.npy'
]

# Load and Explore Data
train_images, train_labels, train_images_third, train_labels_third, val_images, val_labels, test_images, test_labels = load_data(data_paths)
dataset_shapes = {
    "Training": train_images.shape,
    "Validation": val_images.shape,
    "Testing": test_images.shape
}
for key, value in dataset_shapes.items():
    print(f"{key} Data Shape: {value}")

# Visualization Function with Enhanced Structure
def plot_results(history):
    metrics = ['loss', 'accuracy', 'recall', 'auc']
    phases = ['train', 'val']
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 2, i)
        for phase in phases:
            plt.plot(history.history[f'{phase}_{metric}'], label=f'{phase} {metric}')
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
    plt.tight_layout()
    plt.show()


In [5]:
#Explore the dataset again
print ("Number of training samples: " + str(train_images.shape[0]))
print ("A third of training samples: " + str(train_images_third.shape[0]))
print ("Number of validation samples: " + str(val_images.shape[0]))
print ("Number of testing samples: " + str(test_images.shape[0]))
print ("===")
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("A third of train_images shape: " + str(train_images_third.shape))
print ("A third of train_labels shape: " + str(train_labels_third.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

Number of training samples: 25803
A third of training samples: 8601
Number of validation samples: 8602
Number of testing samples: 3823
===
train_images shape: (25803, 128, 128, 3)
train_labels shape: (25803,)
A third of train_images shape: (8601, 128, 128, 3)
A third of train_labels shape: (8601,)
val_images shape: (8602, 128, 128, 3)
val_labels shape: (8602,)
test_images shape: (3823, 128, 128, 3)
test_labels shape: (3823,)


In [6]:
#Build a pretrained InceptionResNet model
inceptionResNet = InceptionResNetV2(weights='imagenet',
                   include_top=False, # Allows us to set input shape
                   input_shape=(128,128,3)) 

2021-11-30 01:28:12.073649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 01:28:12.166390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 01:28:12.167150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 01:28:12.169030: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
#Add inceptionResNet to our model
inceptionResNet_model = keras.models.Sequential()
inceptionResNet_model.add(inceptionResNet)

#Freezing layers so they don't get re-trained with new data
for layer in inceptionResNet_model.layers:
    layer.trainable=False 
    
#Adding our own dense layers
inceptionResNet_model.add(layers.Flatten())
inceptionResNet_model.add(layers.Dense(128, activation='relu'))
inceptionResNet_model.add(layers.Dense(1, activation='sigmoid'))

In [9]:
#Compile inceptionResNet model
inceptionResNet_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'Recall', 'AUC'])

#Instantiate an EarlyStopping Object
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
mc = ModelCheckpoint('best_inceptionResNet_model.h5', monitor='val_recall', mode='max', verbose=1, save_best_only=True)

#And fit the inceptionResNet to the training images, validating on the val images
results = inceptionResNet_model.fit(train_images_third,
                            train_labels_third,
                            epochs=50,
                            batch_size=32,
                            callbacks = [es,mc],
                            validation_data=(val_images, val_labels))

2021-11-30 01:28:22.579953: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1691025408 exceeds 10% of free system memory.
2021-11-30 01:28:24.465836: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1691025408 exceeds 10% of free system memory.
2021-11-30 01:28:25.937225: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2021-11-30 01:28:36.584488: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


269/269 [==============================] - ETA: 0s - loss: 0.2994 - accuracy: 0.9052 - recall: 0.4931 - auc: 0.8730

2021-11-30 01:29:01.668487: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1691222016 exceeds 10% of free system memory.
2021-11-30 01:29:03.642715: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1691222016 exceeds 10% of free system memory.


269/269 [==============================] - 58s 156ms/step - loss: 0.2994 - accuracy: 0.9052 - recall: 0.4931 - auc: 0.8730 - val_loss: 0.2888 - val_accuracy: 0.9070 - val_recall: 0.3980 - val_auc: 0.8840

Epoch 00001: val_recall improved from -inf to 0.39797, saving model to best_inceptionResNet_model.h5
Epoch 2/50
269/269 [==============================] - 33s 122ms/step - loss: 0.2013 - accuracy: 0.9278 - recall: 0.6120 - auc: 0.9306 - val_loss: 0.2466 - val_accuracy: 0.9184 - val_recall: 0.5395 - val_auc: 0.9003

Epoch 00002: val_recall improved from 0.39797 to 0.53948, saving model to best_inceptionResNet_model.h5
Epoch 3/50
269/269 [==============================] - 33s 122ms/step - loss: 0.1755 - accuracy: 0.9363 - recall: 0.6605 - auc: 0.9480 - val_loss: 0.2273 - val_accuracy: 0.9216 - val_recall: 0.5559 - val_auc: 0.9160

Epoch 00003: val_recall improved from 0.53948 to 0.55590, saving model to best_inceptionResNet_model.h5
Epoch 4/50
269/269 [==============================] - 

In [10]:
#Visualize Results 
visualize_results(results)

In [11]:
#Save Weights
inceptionResNet_model.save_weights('inceptionResNet_model_weights.h5')